# Brasileirão Série A: O caminho dos campeões
Por [Daniel Soares](https://www.linkedin.com/in/daniel-soares-ti/)


<div style="display:inline-block;vertical-align:top;">
<h2>Sumário:</h2>
<ul>
    <li><a href="#introducao">Introdução </a></li>
    <li><a href="#maior-campeao">Maiores campeões</a></li>
    <li><a href="#melhor-pior">Melhor e "pior" campeão</a></li>
    <li><a href="#ano-competitivo">Competitividade durante os anos</a></li>
    <li><a href="#tecnicas_metodos">Algum time venceu anos consecutivos?</a></li>

</ul>
</div>
<div style="display:inline-block; margin-left:50px; margin-top:15px">
<img src="https://media.giphy.com/media/xUOxf1JRLpx5wy7UyY/giphy.gif" style="height: 230px; width:370px;"/>
    <div style="text-align:center; font-size:12px"><i>Fonte: <a href="https://media.giphy.com/media/xUOxf1JRLpx5wy7UyY/giphy.gif">Giphy</a></i></div>
</div>

--------


In [2]:
import pandas as pd
import plotly.express as px

# Lendo os dados
games_data = pd.read_csv('../data/campeonato-brasileiro-full.csv')

In [3]:
def corrigir_indexes_e_colunas_tabela(dataframe):
    """Corrige indexes e colunas do dataframe

    Args:
        dataframe (pd.Dataframe): Dataframe com os jogos

    Returns:
        pd.Dataframe: Dataframe com os indexes e colunas corrigidos
    """
    dataframe = dataframe.reset_index()
    dataframe = dataframe.set_index(pd.Index(range(1, dataframe.shape[0] + 1)))
    dataframe = dataframe.rename(columns={'vencedor': 'times', 
                                          'count': 'pontos'})
    
    return dataframe

def inserir_ano_do_titulo(dataframe_pontos, ano):
    """Insere o ano da conquista no

    Args:
        dataframe_pontos (pd.DataFrame): tabela com a classificação
        ano (int): ano para inserir
        
    Returns:
        pd.DataFrame: tabela com o ano da conquista inserido
    """
    dataframe_pontos['ano'] = ano
    
    return dataframe_pontos

def calcular_distancia_segundo_colocado(dataframe_pontos):
    """Calcula a diferença de pontos entre o primeiro e o segundo colocado

    Args:
        dataframe_pontos (pd.DataFrame): tabela com a classificação

    Returns:
        pd.DataFrame: tabela com a diferença de pontos calculada
    """
    dataframe_pontos['distancia_segundo_colocado'] = dataframe_pontos.iloc[0][1]\
        - dataframe_pontos.iloc[1][1]
        
    return dataframe_pontos

def calcular_pontos_de_vitoria(dataframe_original, ano):
    """Calcula a pontuação inicial dos times com base em suas vitórias

    Args:
        dataframe_original (pd.Dataframe): Dataframe com os jogos
        ano (int): ano do campeonato

    Returns:
        pd.Dataframe: Dataframe com os pontos iniciais calculados
    """
    tabela_campeonato = dataframe_original.query(f'ano == {ano}').vencedor\
        .value_counts()[1:].to_frame()
    tabela_campeonato = tabela_campeonato * 3
    
    return tabela_campeonato

def adicionar_pontos_empate(dataframe_original, dataframe_pontos, ano):
    """Adiciona pontos para os jogos que terminaram empatados

    Args:
        dataframe (pd.Dataframe): dataframe com os jogos
        dataframe_pontos (pd.Dataframe): dataframe com os pontos iniciais
        ano (int): ano do campeonato

    Returns:
        pd.Dataframe: dataframe com os pontos de empate inseridos
    """
    for time in dataframe_original.query(f"ano == {ano} and vencedor =='-'")\
        .mandante:
        dataframe_pontos.loc[time] = dataframe_pontos.loc[time] + 1
        
    for time in dataframe_original.query(f"ano == {ano} and vencedor =='-'")\
        .visitante:
        dataframe_pontos.loc[time] = dataframe_pontos.loc[time] + 1
    
    return dataframe_pontos

def aplicar_punicoes_2003(tabela_de_classificacao):
    """Aplica as punições de perda de pontos referente ao ano de 2003

    Args:
        tabela_de_classificacao (pd.DataFrame): tabela com a classificação
        
    Returns:
        pd.DataFrame: tabela com as punições aplicadas
    """
    tabela_de_classificacao.loc['Ponte Preta'] = tabela_de_classificacao\
        .loc['Ponte Preta'] - 1
    tabela_de_classificacao.loc['Internacional'] = tabela_de_classificacao\
        .loc['Internacional'] + 2
    tabela_de_classificacao.loc['Juventude'] = tabela_de_classificacao\
        .loc['Juventude'] + 3
    tabela_de_classificacao.loc['Paysandu'] = tabela_de_classificacao\
        .loc['Paysandu'] - 8
    tabela_de_classificacao.loc['Sao Caetano'] = tabela_de_classificacao\
        .loc['Sao Caetano'] + 3
    tabela_de_classificacao.loc['Corinthians'] = tabela_de_classificacao\
        .loc['Corinthians'] + 2
    tabela_de_classificacao.loc['Fluminense'] = tabela_de_classificacao\
        .loc['Fluminense'] + 2
        
    return tabela_de_classificacao

def aplicar_punicoes_2004(tabela_de_classificacao):
    """Aplica as punições de perda de pontos referente ao ano de 2004

    Args:
        tabela_de_classificacao (pd.DataFrame): tabela com a classificação
    
    Returns:
        pd.DataFrame: tabela com as punições aplicadas
    """
    tabela_de_classificacao.loc['Sao Caetano'] = tabela_de_classificacao\
        .loc['Sao Caetano'] - 24
        
    return tabela_de_classificacao
    

def aplicar_punicoes_2010(tabela_de_classificacao):
    """Aplica as punições de perda de pontos referente ao ano de 2010

    Args:
        tabela_de_classificacao (pd.DataFrame): tabela com a classificação

    Returns:
        pd.DataFrame: tabela com as punições aplicadas
    """
    tabela_de_classificacao.loc['Gremio Prudente'] = tabela_de_classificacao\
        .loc['Gremio Prudente'] - 3
    
    return tabela_de_classificacao

def aplicar_punicoes_2013(tabela_de_classificacao):
    """Aplica as punições de perda de pontos referente ao ano de 2013

    Args:
        tabela_de_classificacao (pd.DataFrame): tabela com a classificação

    Returns:
        pd.DataFrame: tabela com as punições aplicadas
    """
    tabela_de_classificacao.loc['Flamengo'] = tabela_de_classificacao\
        .loc['Flamengo'] - 4
    tabela_de_classificacao.loc['Portuguesa'] = tabela_de_classificacao\
        .loc['Portuguesa'] - 4
    
    return tabela_de_classificacao

def aplicar_punicoes_2016(tabela_de_classificacao):
    """Aplica as punições de perda de pontos referente ao ano de 2016

    Args:
        tabela_de_classificacao (pd.DataFrame): tabela com a classificação

    Returns:
        pd.DataFrame: tabela com as punições aplicadas
    """
    tabela_de_classificacao.loc['Santa Cruz'] = tabela_de_classificacao\
        .loc['Santa Cruz'] - 3
    
    return tabela_de_classificacao

def aplicar_punicoes_2018(tabela_de_classificacao):
    """Aplica as punições de perda de pontos referente ao ano de 2018

    Args:
        tabela_de_classificacao (pd.DataFrame): tabela com a classificação

    Returns:
        pd.DataFrame: tabela com as punições aplicadas
    """
    tabela_de_classificacao.loc['Sport'] = tabela_de_classificacao\
        .loc['Sport'] - 3
    
    return tabela_de_classificacao

def buscar_campeoes(classificacao, lista_de_anos):
    """Busca os campeões de cada ano

    Args:
        classificacao (dict): Dicionário contendo as classificações de cada ano
        lista_de_anos (list): lista contendo os anos que serão buscados

    Returns:
        pd.DataFrame: tabela com os campeões de cada ano
    """
    tabela_campeoes = pd.DataFrame()
    for ano in lista_de_anos:
        tabela_campeoes = pd.concat([tabela_campeoes, 
                                     classificacao[ano].iloc[0, :].to_frame().T]
                                    )
    
    return tabela_campeoes

<a id='introducao'></a>
# Introdução
"O Brasileirão é o campeonato mais difícil do mundo". Essa é uma frase que você 
provavelmente já escutou na rua, e que vem provando ser verdadeira. A Federação 
Internacional de História e Estatísticas do Futebol (IFFHS), organização 
reconhecida pela FIFA, [publicou](https://www.iffhs.com/posts/2483) no início de 
2023 um ranking de campeonatos nacionais, e levando em conta fatores como força 
dos concorrentes e conquistas dos times na temporada, coroou o Brasileirão como 
o mais difícil do mundo pelo segundo ano consecutivo.

Diante disso, chegamos a uma pergunta: Qual o caminho dos campeões? O que um time
precisa para conquistar o atual campeonato mais difícil do planeta?

<div class="alert alert-warning" role="alert">
    <b>Aviso!</b> Esta análise não irá considerar os títulos anteriores ao ano 
    de 2003.
</div>

In [4]:
# Para evitar problemas quando for plotar os gráficos, vou alterar o dtype
# das colunas que necessitam
games_data['data'] = pd.to_datetime(games_data['data'], dayfirst=True)
games_data['hora'] = pd.to_datetime(games_data['hora'], format='%H:%M').dt.time

# Para separar os campeonatos por ano, irei criar uma coluna com o ano 
# de cada jogo. Aqui, irei alterar manualmente os anos de 2020 e 2021, que 
# possuem rodadas de campeonatos diferentes devido a pandemia
games_data['ano'] = games_data['data'].dt.year

indexes_rodadas_2020 = games_data\
    .query("data > '2020-08-01' and data < '2021-03-01'").index
    
games_data.iloc[indexes_rodadas_2020, -1] = 2020


In [7]:
# Para gerar as classificações de forma correta, terei que separar os anos
# que tiveram punições e os que não tiveram para então usar as funções

anos_sem_punicao = [2005, 2006, 2007, 2008, 2009, 2011, 2012, 2014, 2015, 2017, 
                    2019, 2020, 2021, 2022]

anos_com_punicao = [2003, 2004, 2010, 2013, 2016, 2018]

classificacao = {}

# Para os anos sem punição, irei usar as funções criadas para calcular os pontos

for ano in anos_sem_punicao:
    classificacao[ano] = calcular_pontos_de_vitoria(games_data, ano)
    
    classificacao[ano] = adicionar_pontos_empate(games_data, classificacao[ano], 
                                                 ano)
    
    classificacao[ano] = classificacao[ano].sort_values(by='count',
                                                          ascending=False)
    
    classificacao[ano] = corrigir_indexes_e_colunas_tabela(classificacao[ano])
    
    classificacao[ano] = inserir_ano_do_titulo(classificacao[ano], ano)
    
    classificacao[ano] = calcular_distancia_segundo_colocado(classificacao[ano])
    
# Para os anos com punição, irei calcular as pontuações e após isso aplicar as
# punições

for ano in anos_com_punicao:
    classificacao[ano] = calcular_pontos_de_vitoria(games_data, ano)
    
    classificacao[ano] = adicionar_pontos_empate(games_data, classificacao[ano],
                                                 ano)

classificacao[2003] = aplicar_punicoes_2003(classificacao[2003])

classificacao[2004] = aplicar_punicoes_2004(classificacao[2004])

classificacao[2010] = aplicar_punicoes_2010(classificacao[2010])

classificacao[2013] = aplicar_punicoes_2013(classificacao[2013])

classificacao[2016] = aplicar_punicoes_2016(classificacao[2016])

classificacao[2018] = aplicar_punicoes_2018(classificacao[2018])

for ano in anos_com_punicao:
    classificacao[ano] = classificacao[ano].sort_values(by='count',
                                                        ascending=False)
    
    classificacao[ano] = corrigir_indexes_e_colunas_tabela(classificacao[ano])
    
    classificacao[ano] = inserir_ano_do_titulo(classificacao[ano], ano)
    
    classificacao[ano] = calcular_distancia_segundo_colocado(classificacao[ano])
    
# Com as tabelas prontas, basta unir todos os anos em uma lista para então
# buscar os campeões de cada ano
    
anos = anos_sem_punicao + anos_com_punicao
    
tabela_times_campeoes = buscar_campeoes(classificacao, sorted(anos))

# E por último, irei criar uma função para rotular os anos com formatos de 
# competição diferentes, onde houve um número maior de times participantes do
# que o formato atual, de 20 times

func_inserir_formato = lambda ano: 'Formato antigo' if ano in [2003, 2004, 2005]\
    else 'Formato atual'

tabela_times_campeoes['formato'] = tabela_times_campeoes['ano']\
    .apply(func_inserir_formato)

    

In [17]:
tabela_times_campeoes

,times,pontos,ano,distancia_segundo_colocado,formato
1,Cruzeiro,100,2003,13,Formato antigo
1,Santos,89,2004,3,Formato antigo
1,Corinthians,81,2005,3,Formato antigo
1,Sao Paulo,78,2006,9,Formato atual
1,Sao Paulo,77,2007,15,Formato atual
1,Sao Paulo,75,2008,3,Formato atual
1,Flamengo,67,2009,2,Formato atual
1,Fluminense,71,2010,2,Formato atual
1,Corinthians,71,2011,2,Formato atual
1,Fluminense,77,2012,5,Formato atual


<a id='maior-campeao'></a>
# Maiores campeões


In [12]:
count_campeoes = tabela_times_campeoes.times.value_counts().to_frame()
count_campeoes = count_campeoes.reset_index()\
    .rename(columns={'count': 'títulos'})

fig = px.treemap(count_campeoes, path = ['times', 'títulos'], values='títulos')

fig.update_layout(
    title=dict(text="Títulos do Campeonato Brasileiro por time", font=dict(size=25))
)

fig.show()

<a id='melhor-pior'></a>
# Melhor e "pior" campeão


In [110]:
fig = px.scatter(tabela_times_campeoes, 
                 x = 'pontos', 
                 y = 'times', 
                 color = 'formato',
                 color_discrete_sequence=['crimson', 'MidnightBlue'])

fig.update_traces(marker=dict(size=12, 
                              line=dict(width=2, 
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'),
                  opacity = 0.70)

fig.update_layout(title=dict(text="Pontuação final de cada Campeão Brasileiro", 
                             font=dict(size=25)))

fig.add_annotation(x=100, 
                   y='Cruzeiro', 
                   text="Cruzeiro de 2002",
                   showarrow=True,
                   arrowhead=6,
                   bordercolor="darkgray",
                   borderwidth=2,
                   borderpad=4,
                   bgcolor="lightgray",
                   arrowsize= 2.75,
                   arrowcolor='crimson') 

fig.add_annotation(x=90, 
                   y='Flamengo', 
                   text="Flamengo de 2019",
                   showarrow=True,
                   arrowhead=6,
                   bordercolor="darkgray",
                   borderwidth=2,
                   borderpad=4,
                   bgcolor="lightgray",
                   arrowsize= 2.75,
                   arrowcolor='MidnightBlue')

fig.add_annotation(x=67, 
                   y='Flamengo', 
                   text="Flamengo de 2009",
                   showarrow=True,
                   arrowhead=6,
                   bordercolor="darkgray",
                   borderwidth=2,
                   borderpad=4,
                   bgcolor="lightgray",
                   arrowsize= 2.75,
                   arrowcolor='MidnightBlue')


<a id='ano-competitivo'></a>
# Competitividade durante os anos

In [122]:
fig = px.line(tabela_times_campeoes, x='ano', 
              y = 'distancia_segundo_colocado',
              text='distancia_segundo_colocado')

fig.update_traces(textposition="bottom left")

fig.update_layout(title=dict(text="Diferença de pontos entre o primeiro e segundo colocado", 
                             font=dict(size=25)))